In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
from sympy import Matrix

filename = Path("../img/color/Jayhawk_512x512.jpg")

with Image.open(filename, "r") as im:
    display(im)

# True for noisy execution, False for noiseless execution
noisy_execution = False
draw_qc = True

In [ ]:
def display_quantum_circuit(qc):
    if draw_qc:
        display(qc.draw('mpl', reverse_bits=True))

In [ ]:
from qcc.filters import *

stride = 1
num_layers = 2
# fltr_raw = avg_filter(2, dim = 2)
# fltr_raw = sobel_filter(3, axis=0)
fltr_raw = laplacian_approx(3)
# fltr_raw = gaussian_blur(5, 1.2)

npad = tuple((0, 2 ** int(np.ceil(np.log2(N))) - N) for N in fltr_raw.shape)
fltr = np.pad(fltr_raw, pad_width=npad, mode="constant", constant_values=0)

x = np.min(np.abs(fltr_raw))
F = fltr_raw / x
print(x)
Matrix(F)

In [ ]:
from qcc.quantum import flatten_image, normalize, wires_to_qubits, to_qubits
import pennylane as qml

psi, *dims = flatten_image(filename, pad=True)
psi, mag = normalize(psi, include_magnitude=True)

_, fltr_mag = normalize(fltr.flatten(order="F"), include_magnitude=True)
fltr /= fltr_mag

num_states = 2**sum(to_qubits(dims))
num_ancilla = sum(to_qubits(fltr.shape))

In [ ]:
from pennylane.templates import AmplitudeEmbedding
from qcc.quantum.operation import Convolution
from qcc.quantum.operation.ansatz.convolution.v1 import ConvolutionAnsatz

module = ConvolutionAnsatz.from_dims(dims, num_layers=1)

ansatz.draw(decompose=True)

In [ ]:
psi_out = np.sqrt(ansatz.qnode(psi, fltr.flatten()))

In [ ]:
from qcc.quantum import construct_img

i = 0
img = psi_out.numpy()[i*num_states:(i+1)*num_states]
norm = mag * fltr_mag * np.sqrt(2**num_ancilla)
img = construct_img(norm*img, dims)
img.save("output.png")

display(img)

In [ ]:
from qcc.filters import convolution

with Image.open(filename, "r") as im:
    image_data = np.asarray(im, dtype=float)

classical_img = convolution(image_data, fltr_raw)
classical_img = Image.fromarray(classical_img.astype(np.uint8))
display(classical_img)
classical_img.save("output_classical.png")

In [ ]:
from qcc.quantum import get_fidelity

fid = get_fidelity(np.asarray(classical_img).flatten(), np.asarray(img).flatten())

print(f"Fidelity: {fid:.03%}")